In [55]:
# Importing necessary libraries
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import pickle
import random
import torch.nn as nn

# Set seeds for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Define the RNN model class (consistent with the earlier model)
class RNNModel(nn.Module):
    def __init__(self, input_size=14, num_layers=2):  # Input size 15 and 2 layers as in the first code block
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size=64, num_layers=num_layers, dropout=0.2, batch_first=True)  # Hidden size 64, dropout 0.2
        self.fc = nn.Sequential(
            nn.Linear(64, 32),  # First fully connected layer
            nn.ReLU(),
            nn.Linear(32, 16),  # Second fully connected layer
            nn.ReLU(),
            nn.Linear(16, 5),   # Final layer with 5 output classes
            nn.Dropout(0.1)     # Dropout for regularization
        )

    def forward(self, x):
        x, _ = self.rnn(x)  # RNN output
        return self.fc(x[:, -1, :])  # Fully connected layers using the last time step's output

# Model parameters
input_size = 14  # Updated to 15, to match the first block
num_layers = 2  # Number of RNN layers
num_classes = 5  # Number of output classes (e.g., 5 plates)

# Instantiate the model (consistent with the earlier model)
model = RNNModel(input_size=input_size, num_layers=num_layers)

# Load state_dict with strict=False to avoid mismatch issues (if applicable)
try:
    model.load_state_dict(torch.load('/content/rnn_trained_model (1).pth'), strict=False)
except RuntimeError as e:
    print(f"Error loading model state_dict: {e}")

# Set model to evaluation mode
model.eval()

# Function to process input data
def process_data(traj):
    """
    Process the input trajectory data for the model.

    Input:
        traj: trajectory data containing longitude, latitude, time, and status.

    Output:
        data: processed data as a tensor suitable for model input.
    """
    # Convert trajectory to DataFrame
    df = pd.DataFrame(traj, columns=['longitude', 'latitude', 'time', 'status'])

    # Convert 'time' to datetime and extract features
    df['time'] = pd.to_datetime(df['time'])
    time_features = ['month', 'day', 'dayofweek', 'hour', 'minute', 'second']
    for feature in time_features:
        df[feature] = getattr(df['time'].dt, feature)
        df[f'{feature}_sin'] = np.sin(2 * np.pi * df[feature] / df[feature].max())
        df[f'{feature}_cos'] = np.cos(2 * np.pi * df[feature] / df[feature].max())

    # Drop original time column and status
    df = df.drop(columns=['time', 'status'] + time_features)

    # Normalize longitude and latitude
    scaler = StandardScaler()
    df[['longitude', 'latitude']] = scaler.fit_transform(df[['longitude', 'latitude']])

    # Convert DataFrame to tensor
    data = torch.FloatTensor(df.values).unsqueeze(0)  # Add batch dimension

    return data

# Function to run the model and get predictions
def run(data, model):
    """
    Get the predicted label from the model for the processed data.

    Input:
        data: the output of the process_data function.
        model: your trained model.

    Output:
        prediction: the predicted label (plate) of the data, an int value.
    """
    with torch.no_grad():  # Disable gradient calculations for inference
        output = model(data)

        # Get the predicted label (plate) as an int value
        prediction = output.argmax(dim=1).item()  # Assuming classification

    return prediction

# Load the test data from 'test.pkl'
with open('/content/test.pkl', 'rb') as f:
    test_data = pickle.load(f)

# Evaluate model on test data
results = []
for trajectory in test_data:
    processed_data = process_data(trajectory)
    prediction = run(processed_data, model)
    results.append(prediction)

# Print results
print("Predictions for test trajectories:")
for i, pred in enumerate(results):
    print(f"Trajectory {i+1}: Predicted Driver {pred}")


Predictions for test trajectories:
Trajectory 1: Predicted Driver 0


<ipython-input-55-a2181916bdf8>:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/rnn_trained_model (1).pth'), strict=False)
